# GPT-4o-mini을 활용한 금융 뉴스 감정 분석

# Library

In [7]:
import os, time, json, csv, hashlib
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
import numpy as np
from datetime import datetime
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
# ===== 파일 경로 =====
INPUT_CSV  = "naver_finance_news_2020_2024_fx_only.csv"
OUTPUT_CSV = "naver_finance_news_2020_2024_fx_only_with_sentiment.csv"  

# ===== OpenAI 설정 =====
API_KEY = os.getenv("OPENAI_API_KEY")
MODEL_ID = "gpt-4o-mini"  
TIMEOUT  = 60
client   = OpenAI(api_key=API_KEY)

# ===== 컬럼 =====
TEXT_COL = "summary_kobart_v3"   # 감정분석 대상(요약문)
COL_SENT = "sentiment_v1"        # Positive / Negative / Neutral
COL_CONF = "sentiment_conf_v1"   # 0~1 (신뢰도)

# ===== 배치/재시도/부분 저장 =====
BATCH_SIZE    = 25
RETRY_MAX     = 5
RETRY_BACKOFF = 2.0
CHUNK_FLUSH   = 200  

# ===== 키 생성(체크포인트/중복 방지) =====
def _row_key(row: dict) -> str:
    s = f"{row.get('date','')}\t{row.get('title','')}\t{row.get('url','')}\t{row.get('content','')}"
    return hashlib.md5(s.encode("utf-8")).hexdigest()

# ===== 완료 여부 판정(엄격) =====
def _is_filled(x) -> bool:
    if pd.isna(x):
        return False
    s = str(x).strip().lower()
    return s not in ("", "nan", "none", "null")

# ===== 프롬프트 (원화 기준 원달러 투자자 관점) =====
SYSTEM_PROMPT = """
You are a concise FX sentiment annotator from the perspective of a KRW investor.
Classify Korean news summaries into:
- Positive : favorable to KRW appreciation (USDKRW ↓)
- Negative : favorable to KRW depreciation (USDKRW ↑)
- Neutral  : unclear/mixed/informational

Return JSON only:
{"sentiment":"Positive|Negative|Neutral","confidence":0.xx}

Rules:
- Always reason in USDKRW terms.
- If mixed, choose Neutral unless one side clearly dominates.
- Output JSON only, no extra text.
"""

FEWSHOT = [
    # Positive (KRW 강세 = USDKRW 하락)
    {"role":"user","content":"요약문:\n미국 물가 둔화로 연준 긴축 완화 기대가 커지며 위험선호가 회복, 원/달러 환율은 하락 압력을 받았다.\n\nJSON 한 객체만 출력하세요."},
    {"role":"assistant","content":'{"sentiment":"Positive","confidence":0.88}'},
    # Negative (KRW 약세 = USDKRW 상승)
    {"role":"user","content":"요약문:\n연준 추가 긴축 우려와 지정학 리스크로 위험회피가 확대되며 원/달러 환율이 1,300원을 재돌파했다.\n\nJSON 한 객체만 출력하세요."},
    {"role":"assistant","content":'{"sentiment":"Negative","confidence":0.90}'},
    # Neutral
    {"role":"user","content":"요약문:\n정부는 외환시장 동향을 점검했고 구체적 조치는 없다고 밝혔다.\n\nJSON 한 객체만 출력하세요."},
    {"role":"assistant","content":'{"sentiment":"Neutral","confidence":0.75}'},
]

def _extract_json(txt: str) -> str:
    if not isinstance(txt, str): return "{}"
    a, b = txt.find("{"), txt.rfind("}")
    return txt[a:b+1] if (a != -1 and b != -1 and b > a) else txt

def call_llm(summary: str):
    delay = RETRY_BACKOFF
    for attempt in range(1, RETRY_MAX+1):
        try:
            resp = client.chat.completions.create(
                model=MODEL_ID,
                temperature=0.0,
                max_tokens=60,
                response_format={"type":"json_object"},
                timeout=TIMEOUT,
                messages=[{"role":"system","content":SYSTEM_PROMPT}] + FEWSHOT + [
                    {"role":"user","content":f"요약문:\n{summary}\n\nJSON 한 객체만 출력하세요."}
                ]
            )
            txt = resp.choices[0].message.content.strip()
            try:
                data = json.loads(txt)
            except json.JSONDecodeError:
                data = json.loads(_extract_json(txt))

            sent = str(data.get("sentiment","Neutral")).strip()
            if sent not in ("Positive","Negative","Neutral"):
                sent = "Neutral"
            conf = float(data.get("confidence",0.0))
            conf = max(0.0, min(1.0, conf))
            return sent, conf

        except Exception as e:
            is429 = ("429" in str(e)) or ("too many requests" in str(e).lower())
            sleep_s = 5.0 if is429 else delay
            print(f"[retry] {attempt}/{RETRY_MAX} err={e} → sleep {sleep_s:.1f}s", flush=True)
            time.sleep(sleep_s)
            delay = min(delay*2.0, 60.0)
    return "Neutral", 0.0  

In [11]:
# 1) 입력 로드
src = pd.read_csv(INPUT_CSV, encoding="utf-8-sig", engine="python")
src = src.rename(columns={c: c.strip() for c in src.columns})
assert TEXT_COL in src.columns, f"'{TEXT_COL}' 컬럼이 필요합니다."

# 2) 키 생성 (입력)
if "_key" not in src.columns:
    src["_key"] = src.apply(lambda r: _row_key({
        "date": r.get("date",""),
        "title": r.get("title",""),
        "url": r.get("url",""),
        "content": r.get("content",""),
    }), axis=1)

# 3) 출력/체크포인트 스켈레톤 보장
if not os.path.exists(OUTPUT_CSV):
    base = src.copy()
    if COL_SENT not in base.columns: base[COL_SENT] = ""
    if COL_CONF not in base.columns: base[COL_CONF] = 0.0
    base.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig",
                quoting=csv.QUOTE_ALL, lineterminator="\n")
    print(f"[INIT] 출력 파일 생성: {OUTPUT_CSV}")

# 4) 기존 체크포인트 로드 + 완료/미완료 분리
out = pd.read_csv(OUTPUT_CSV, encoding="utf-8-sig", engine="python")
out = out.rename(columns={c: c.strip() for c in out.columns})

if "_key" not in out.columns:
    out["_key"] = out.apply(lambda r: _row_key({
        "date": r.get("date",""),
        "title": r.get("title",""),
        "url": r.get("url",""),
        "content": r.get("content",""),
    }), axis=1)

if COL_SENT not in out.columns: out[COL_SENT] = ""
if COL_CONF not in out.columns: out[COL_CONF] = 0.0

filled_mask = out[COL_SENT].apply(_is_filled)
done_keys    = set(out.loc[filled_mask, "_key"])
todo_keys    = set(src["_key"]) - done_keys

todo_df = src[src["_key"].isin(todo_keys)].copy()
print(f"[INFO] 전체 {len(src)}행, 완료 {len(done_keys)}행, 신규 분석 대상 {len(todo_df)}행")

# 5) 라벨링 루프 (미완료만) + 부분 저장
buffer = []
with tqdm(total=len(todo_df), desc="Sentiment labelling (resume)") as pbar:
    for _, r in todo_df.iterrows():
        summ = str(r.get(TEXT_COL,"") or "").strip()
        if not summ:
            sent, conf = "Neutral", 0.0
        else:
            sent, conf = call_llm(summ)

        buffer.append((r["_key"], sent, conf))
        pbar.update(1)

        if len(buffer) >= CHUNK_FLUSH:
            out = pd.read_csv(OUTPUT_CSV, encoding="utf-8-sig", engine="python")
            if "_key" not in out.columns:
                out["_key"] = out.apply(lambda rr: _row_key({
                    "date": rr.get("date",""),
                    "title": rr.get("title",""),
                    "url": rr.get("url",""),
                    "content": rr.get("content",""),
                }), axis=1)
            upd = {k:(s,c) for k,s,c in buffer}
            m = out["_key"].isin(upd.keys())
            for i in out.index[m]:
                k = out.at[i, "_key"]; s,c = upd[k]
                out.at[i, COL_SENT] = s
                out.at[i, COL_CONF] = c
            out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig",
                       quoting=csv.QUOTE_ALL, lineterminator="\n")
            buffer = []

# 남은 것 반영
if buffer:
    out = pd.read_csv(OUTPUT_CSV, encoding="utf-8-sig", engine="python")
    if "_key" not in out.columns:
        out["_key"] = out.apply(lambda rr: _row_key({
            "date": rr.get("date",""),
            "title": rr.get("title",""),
            "url": rr.get("url",""),
            "content": rr.get("content",""),
        }), axis=1)
    upd = {k:(s,c) for k,s,c in buffer}
    m = out["_key"].isin(upd.keys())
    for i in out.index[m]:
        k = out.at[i, "_key"]; s,c = upd[k]
        out.at[i, COL_SENT] = s
        out.at[i, COL_CONF] = c
    out.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig",
               quoting=csv.QUOTE_ALL, lineterminator="\n")

print(f"[DONE] 저장: {OUTPUT_CSV}")

[INFO] 전체 30076행, 완료 20225행, 신규 분석 대상 9851행


Sentiment labelling (resume): 100%|██████████████████████████████████████████████| 9851/9851 [2:11:04<00:00,  1.25it/s]


[DONE] 저장: naver_finance_news_2020_2024_fx_only_with_sentiment.csv


In [12]:
df = pd.read_csv(OUTPUT_CSV, encoding="utf-8-sig", engine="python")

# 분포 확인
print(df["sentiment_v1"].value_counts(dropna=False))

# 점수화(+1/0/-1) 및 일자별 집계 저장
if "date" in df.columns:
    def _parse_date_any(x):
        s = str(x).strip()
        for fmt in ("%Y-%m-%d","%Y.%m.%d","%Y/%m/%d","%Y%m%d"):
            try:
                return datetime.strptime(s, fmt).date().isoformat()
            except Exception:
                pass
        try:
            return pd.to_datetime(s).date().isoformat()
        except Exception:
            return s
    df["date"] = df["date"].apply(_parse_date_any)

score_map = {"Positive":1, "Neutral":0, "Negative":-1}
df["sentiment_score_v1"] = df["sentiment_v1"].map(score_map).fillna(0).astype(int)

if "date" in df.columns:
    daily = df.groupby("date").agg(
        n=("sentiment_v1","count"),
        pos=("sentiment_v1", lambda x: (x=="Positive").sum()),
        neu=("sentiment_v1", lambda x: (x=="Neutral").sum()),
        neg=("sentiment_v1", lambda x: (x=="Negative").sum()),
        score_mean=("sentiment_score_v1","mean"),
        score_sum=("sentiment_score_v1","sum"),
        conf_mean=("sentiment_conf_v1","mean"),
    ).reset_index()
    daily.to_csv("daily_fx_only_sentiment_index.csv", index=False, encoding="utf-8-sig")
    print("[DONE] 일자별 집계 저장: daily_fx_only_sentiment_index.csv")

sentiment_v1
Neutral     13296
Negative     9385
Positive     7395
Name: count, dtype: int64
[DONE] 일자별 집계 저장: daily_fx_only_sentiment_index.csv


In [13]:
# 1) 거시 변수 데이터
df_macro = pd.read_csv("df_0820_final.csv")
df_macro["date"] = pd.to_datetime(df_macro["date"])

# 2) 일자별 감정 분석 결과 
df_sent_daily = pd.read_csv("daily_fx_only_sentiment_index.csv")
df_sent_daily["date"] = pd.to_datetime(df_sent_daily["date"])

# 3) 날짜 기준 병합
df_final = pd.merge(df_macro, df_sent_daily, on="date", how="left")

# 4) 저장
df_final.to_csv("df_0820_final_with_sentiment.csv", index=False, encoding="utf-8-sig")

print("[DONE] 최종 저장: df_0820_final_with_sentiment.csv")

[DONE] 최종 저장: df_0820_final_with_sentiment.csv


In [9]:
df_sent_daily = pd.read_csv("naver_finance_news_2020_2024_fx_only_with_sentiment.csv")
df_sent_daily

,date,title,url,content,category,confidence,year,summary_kobart_v3,sentiment_v1,sentiment_conf_v1,_key
0,2020-01-01,"외환당국, 3분기 28억7천만달러 순매도…시장안정조치(종합)",https://n.news.naver.com/mnews/article/001/001...,달러 외환당국이 지난 3분기(7~9월) 시장안정을 위해 외환시장에서 28억7천만달러...,FX-Direct,0.95,2020,달러 외환당국이 지난 3분기(7~9월) 시장안정을 위해 외환시장에서 28억7천만달러...,Neutral,0.70,d753a0c3e243ab23bd9af6574e0cd743
1,2020-01-01,"[1보] 외환당국, 3분기 중 28억7천만달러 순매도",https://n.news.naver.com/mnews/article/001/001...,"한국은행·기재부, 외환시장 개입내역 공개달러",FX-Direct,0.95,2020,"한국은행·기재부, 외환시장 개입내역 공개달러",Neutral,0.70,d4120edf9f2cf479f684121cd216fdb4
2,2020-01-01,2019년 달러 대비 엔환율 변동폭 6.82엔…1998년 이후 최소,https://n.news.naver.com/mnews/article/001/001...,박세진 특파원 = 도쿄 외환 시장에서 미 달러화 대비 엔화 환율의 등락폭이 올해 2...,FX-Indirect,0.85,2020,도쿄 외환 시장에서 미 달러화 대비 엔화 환율의 등락폭이 올해 20여년 만에 가장 ...,Neutral,0.70,0b94cd19d32e7c8b8803a2e1f22990e0
3,2020-01-01,브라질 증시 올해 32% 올라…내년에도 상승세 이어갈 듯,https://n.news.naver.com/mnews/article/001/001...,헤알화 가치는 3.5% 하락 김재순 특파원 = 브라질 상파울루 증시가 올해 들어 3...,FX-Direct,0.95,2020,"30일 브라질 상파울루 증시의 보베스파 지수는 전날보다 0.76% 떨어진 115, ...",Neutral,0.70,b9b40a3ff6724ad4762a69a868965522
4,2020-01-01,올해 원/달러 환율 롤러코스터…연중 변동폭 110원 달해,https://n.news.naver.com/mnews/article/001/001...,"내년 연평균 원/달러 환율 1, 165원 안팎 전망 많아30일 오전 코스피지수와 원...",FX-Direct,0.95,2020,"국내 성장세가 낮아진 가운데 미·중 무역분쟁, 일본 수출규제, 홍콩 시위 등 재료가...",Neutral,0.78,7f13225e2b799490cf121aa9c877e992
...,...,...,...,...,...,...,...,...,...,...,...
30071,2024-12-31,달러 강세 계속,https://n.news.naver.com/mnews/article/001/001...,"원/달러 환율 1, 470원대 중반에서 거래가 이어지고 있는 30일 서울 명동 환전...",FX-Direct,0.95,2024,"원/달러 환율이 1, 470원대 중반에서 거래가 이어지고 있는 30일 서울 명동 환...",Neutral,0.80,aee764dbbe50d0627477b356531296eb
30072,2024-12-31,"환율 종가 1,472.5원…연말 기준 외환위기 후 27년 만에 최고",https://n.news.naver.com/mnews/article/001/001...,올해 원/달러 환율 연말 주간 거래 종가가 외환위기였던 1997년 이후 가장 높은 ...,FX-Direct,0.95,2024,30일 서울 외환시장에서 미국 달러화 대비 원화 환율의 주간 거래 종가는 전 거래일...,Negative,0.85,5143cfe218825da9c116014138c00d69
30073,2024-12-31,"환율 종가 1,472.5원…연말 기준 외환위기 후 27년 만에 최고",https://n.news.naver.com/mnews/article/001/001...,올해 원/달러 환율 연말 주간 거래 종가가 외환위기였던 1997년 이후 가장 높은 ...,FX-Direct,0.95,2024,30일 서울 외환시장에서 미국 달러화 대비 원화 환율의 주간 거래 종가는 전 거래일...,Negative,0.85,bbed92bc6828b26d929c9cafc02eb229
30074,2024-12-31,"환율 종가 1,472.5원…연말 기준 외환위기 후 27년 만에 최고",https://n.news.naver.com/mnews/article/001/001...,올해 원/달러 환율 연말 주간 거래 종가가 외환위기였던 1997년 이후 가장 높은 ...,FX-Direct,0.95,2024,30일 서울 외환시장에서 미국 달러화 대비 원화 환율의 주간 거래 종가는 전 거래일...,Negative,0.85,0f2b1a04c7b81434b938bbf9b85f64d9


In [4]:
# ===== 감정 점수 매핑(+1/0/-1) 및 날짜 변환 =====
score_map = {'Positive': 1, 'Neutral': 0, 'Negative': -1}

# 컬럼 존재 확인(이름이 다르면 여기서 맞춰주세요)
required_cols = {'date', 'category', 'sentiment_v1'}
missing = required_cols - set(df_sent_daily.columns)
if missing:
    raise KeyError(f"다음 컬럼이 필요합니다: {missing}")

df = df_sent_daily.copy()
df['sentiment_score'] = df['sentiment_v1'].map(score_map)
df['date'] = pd.to_datetime(df['date'])

# ===== Helper: 일자별 평균 감정 + 뉴스 수 =====
def daily_stats(sub):
    return (
        sub.groupby('date', as_index=True)
           .agg(sentiment_mean=('sentiment_score', 'mean'),
                news_count=('sentiment_score', 'count'))
    )

# ===== 3가지 케이스 산출 =====
direct_df   = daily_stats(df[df['category'] == 'FX-Direct']).add_prefix('Direct_')
indirect_df = daily_stats(df[df['category'] == 'FX-Indirect']).add_prefix('Indirect_')
combined_df = daily_stats(df[df['category'].isin(['FX-Direct', 'FX-Indirect'])]).add_prefix('Combined_')

# ===== 병합 및 후처리 =====
sentiment_daily = (
    pd.concat([direct_df, indirect_df, combined_df], axis=1)
      .sort_index()
)

# 결측치 처리(해당 일자에 뉴스가 없던 케이스)
# 평균은 0으로, 카운트는 0으로
fill_map = {
    'Direct_sentiment_mean': 0.0, 'Direct_news_count': 0,
    'Indirect_sentiment_mean': 0.0, 'Indirect_news_count': 0,
    'Combined_sentiment_mean': 0.0, 'Combined_news_count': 0,
}
sentiment_daily = sentiment_daily.fillna(value=fill_map)

# 소수점 정리(선택)
sentiment_daily['Direct_sentiment_mean'] = sentiment_daily['Direct_sentiment_mean'].round(4)
sentiment_daily['Indirect_sentiment_mean'] = sentiment_daily['Indirect_sentiment_mean'].round(4)
sentiment_daily['Combined_sentiment_mean'] = sentiment_daily['Combined_sentiment_mean'].round(4)

# 인덱스(date) 컬럼으로 빼기
sentiment_daily = sentiment_daily.reset_index()

# ===== 저장 =====
out_path = "fx_news_daily_sentiment_1010.csv"
sentiment_daily.to_csv(out_path, index=False, encoding='utf-8-sig')

print(f"저장 완료: {out_path}")
print(sentiment_daily.head())

저장 완료: fx_news_daily_sentiment_1010.csv
        date  Direct_sentiment_mean  Direct_news_count  \
0 2020-01-01                 0.1000                 10   
1 2020-01-02                 0.2000                 10   
2 2020-01-03                -0.1667                 12   
3 2020-01-04                -0.2222                  9   
4 2020-01-05                -0.3333                  9   

   Indirect_sentiment_mean  Indirect_news_count  Combined_sentiment_mean  \
0                   0.0000                  2.0                   0.0833   
1                   0.0000                  1.0                   0.1818   
2                   0.0000                  1.0                  -0.1538   
3                  -0.3333                  3.0                  -0.2500   
4                  -0.3333                  3.0                  -0.3333   

   Combined_news_count  
0                   12  
1                   11  
2                   13  
3                   12  
4                   12  
